# 原理部分

## ID3 和 C4.5

之前在 [人工智能导论](https://dropsong.github.io/posts/6f3f8819.html) 中 ID3 决策树算法信息增益的方法有一个问题：

在图片 80-26 中，可以看到 ``信息增益 = 信息熵 - 条件熵``，这里的条件熵在计算的时候，会乘以一个系数（特征取值数的倒数）。这样在实践中，会造成**偏向特征取值数较多的特征**。

为了解决这个问题，C4.5 算法提出了**信息增益比最大**的原则。

ID3 决策树：

$$
Gain(D, a) = Ent(D) - \sum_{i}^{k} p_i \cdot Ent(D | i)
$$

C4.5 决策树提出了信息增益率：

$$
GainRatio(D, a) = \frac{Gain(D, a)}{Ent(a)}
$$

<br>

| 名称  |  分支方式   | 备注                                                                 |
|-------|------------|----------------------------------------------------------------------|
| ID3   | 信息增益   | ID3只能对离散属性的数据集构造决策树                                   |
| C4.5  | 信息增益率 | 优化后解决了ID3分支过程中总喜欢偏向取值较多的属性                       |
| CART  | Gini系数   | 可以进行分类和回归，可以处理离散属性，也可以处理连续的               |

有益的补充：
- [决策树（上）——ID3、C4.5、CART（非常详细）](https://zhuanlan.zhihu.com/p/85731206)，该内容已备份为 pdf，在该 ipynb 文件的 github 仓库同级目录下。
- [决策树--信息增益，信息增益比，Geni指数的理解](https://www.cnblogs.com/muzixi/p/6566803.html)，该文章已在 [archive.org](https://archive.org/) 中备份。

一些更具体的细节可以参考 [《统计学习方法(李航)》](https://github.com/qqqil/books/blob/master/%E8%AE%A1%E7%AE%97%E6%9C%BA%E2%97%8F%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%E2%97%8F%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0/%E7%BB%9F%E8%AE%A1%E5%AD%A6%E4%B9%A0%E6%96%B9%E6%B3%95(%E6%9D%8E%E8%88%AA).pdf) 。

ID3 的缺点：
- ID3 没有剪枝策略，容易过拟合；
- 信息增益准则对可取值数目较多的特征有所偏好，类似“编号”的特征其信息增益接近于 1；
- 只能用于处理离散分布的特征；
- 没有考虑缺失值。

C4.5 相对于 ID3 的缺点对应有以下改进方式：
- 引入悲观剪枝策略进行后剪枝；
- 引入信息增益率作为划分标准；
- 将连续特征离散化，假设 n 个样本的连续特征 A 有 m 个取值，C4.5 将其排序并取相邻两样本值的平均数共 m-1 个划分点，分别计算以该划分点作为二元分类点时的信息增益，并选择信息增益最大的点作为该连续特征的二元离散分类点；
- 对于缺失值的处理可以分为两个子问题：
- 问题一：在特征值缺失的情况下进行划分特征的选择？（即如何计算特征的信息增益率）
- 问题二：选定该划分特征，对于缺失该特征值的样本如何处理？（即到底把这个样本划分到哪个结点里）
- 针对问题一，C4.5 的做法是：对于具有缺失值特征，用没有缺失的样本子集所占比重来折算；
- 针对问题二，C4.5 的做法是：将样本同时划分到所有子节点，不过要调整样本的权重值，其实也就是以不同概率划分到不同节点中。

C4.5 的缺点：
- 剪枝策略可以再优化；
- C4.5 用的是多叉树，用二叉树效率更高；
- C4.5 只能用于分类；
- C4.5 使用的熵模型拥有大量耗时的对数运算，连续值还有排序运算；
- C4.5 在构造树的过程中，对数值属性值需要按照其大小进行排序，从中选择一个分割点，所以只适合于能够驻留于内存的数据集，当训练集大得无法在内存容纳时，程序无法运行。

## CART 算法流程

CART(Classification and Regression Tree) 决策树使用"基尼指数" (Gini index)来选择划分属性。

基尼值 Gini（D）：从数据集 D 中随机抽取两个样本，其类别标记不一致的概率。故，**Gini（D）值越小，数据集 D 的纯度越高**。

数据集 D 的纯度可用基尼值来度量:

$$
Gini(D) = \sum_{k=1}^{|y|} \sum_{k' \ne k} p_k p_{k'} = 1 - \sum_{k=1}^{|y|} p_k^2
$$

基尼指数 Gini_index(D)：一般选择使划分后基尼系数最小的属性作为最优化分属性。

$$
GiniIndex(D, a) = \sum_{v=1}^V \frac{D^v}{D} Gini(D^v)
$$

CART 的算法流程：

```
while(当前节点"不纯")：
    1.遍历每个变量的每一种分割方式，找到最好的分割点
    2.分割成两个节点N1和N2
end while
每个节点足够“纯”为止
```

案例，根据下图列表，按照基尼指数的划分依据，做出决策树。

| 序号 | 是否有房 | 婚姻状况  | 年收入 | 是否拖欠贷款 |
|----|------|-------|------|--------|
| 1  | yes  | single| 125k | no     |
| 2  | no   | married|100k | no     |
| 3  | no   | single| 70k  | no     |
| 4  | yes  | married|120k | no     |
| 5  | no   | divorced|95k | yes    |
| 6  | no   | married|60k  | no     |
| 7  | yes  | divorced|220k | no     |
| 8  | no   | single| 85k  | yes    |
| 9  | no   | married|75k  | no     |
| 10 | No   | Single |90k  | Yes    |

整理：

![pic1](./1.png)

step 1，对数据集非序列标号属性 ``{是否有房，婚姻状况，年收入}`` 分别计算它们的 Gini 指数，取 Gini 指数最小的属性作为决策树的根节点属性。

> 第一次大循环

step 2，根节点的Gini值为：

$$
Gini(\text{是否拖欠贷款}) = 1 - (\frac{3}{10})^2 - (\frac{7}{10})^2 = 0.42
$$

step 3，当根据是否有房来进行划分时，Gini指数计算过程为：

$$
Gini(左子节点) = 1 - (\frac{0}{3})^2 - (\frac{3}{3})^2 = 0
$$

$$
Gini(右子节点) = 1 - (\frac{3}{7})^2 - (\frac{4}{7})^2 = 0.4898
$$

$$
GiniIndex(D, \text{是否有房}) = \frac{7}{10} \cdot 0.4898 + \frac{3}{10} \cdot 0 = 0.343
$$

![pic2](./2.png)

![pic3](./3.png)

![pic4](./4.png)

# sklearn 决策树 API

```python
class sklearn.tree.DecisionTreeClassifier(*, criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, ccp_alpha=0.0, monotonic_cst=None)
```

参考官网：
https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

criterion : ``{“gini”, “entropy”, “log_loss”}, default=”gini”``
- The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “log_loss” and “entropy” both for the Shannon information gain, see [Mathematical formulation](https://scikit-learn.org/stable/modules/tree.html#tree-mathematical-formulation).
- 可以是 id3, 或者 c4.5

max_depth: 树的深度大小。

random_state: 随机数种子。

min_samples_split, int 或 float，默认为 2 .


# 决策树实例

接下来我们**举一个例子，泰坦尼克号数据**。

这个数据集描述泰坦尼克号上的个别乘客的生存状态。数据集的特征是票的类别，存活，乘坐班，年龄，登陆，home.dest，房间，票，船和性别等。

In [1]:
import pandas as pd
titan = pd.read_csv("./data/titanic.csv")
titan

,row.names,pclass,survived,name,age,embarked,home.dest,room,ticket,boat,sex
0,1,1st,1,"Allen, Miss Elisabeth Walton",29.0000,Southampton,"St Louis, MO",B-5,24160 L221,2,female
1,2,1st,0,"Allison, Miss Helen Loraine",2.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
2,3,1st,0,"Allison, Mr Hudson Joshua Creighton",30.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,(135),male
3,4,1st,0,"Allison, Mrs Hudson J.C. (Bessie Waldo Daniels)",25.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
4,5,1st,1,"Allison, Master Hudson Trevor",0.9167,Southampton,"Montreal, PQ / Chesterville, ON",C22,NaN,11,male
...,...,...,...,...,...,...,...,...,...,...,...
1308,1309,3rd,0,"Zakarian, Mr Artun",NaN,NaN,NaN,NaN,NaN,NaN,male
1309,1310,3rd,0,"Zakarian, Mr Maprieder",NaN,NaN,NaN,NaN,NaN,NaN,male
1310,1311,3rd,0,"Zenn, Mr Philip",NaN,NaN,NaN,NaN,NaN,NaN,male
1311,1312,3rd,0,"Zievens, Rene",NaN,NaN,NaN,NaN,NaN,NaN,female


In [2]:
x = titan[['pclass', 'age', 'sex']]
y = titan['survived']

x.info() # 看看有没有空值
print('================')
print(x)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  1313 non-null   object 
 1   age     633 non-null    float64
 2   sex     1313 non-null   object 
dtypes: float64(1), object(2)
memory usage: 30.9+ KB
     pclass      age     sex
0       1st  29.0000  female
1       1st   2.0000  female
2       1st  30.0000    male
3       1st  25.0000  female
4       1st   0.9167    male
...     ...      ...     ...
1308    3rd      NaN    male
1309    3rd      NaN    male
1310    3rd      NaN    male
1311    3rd      NaN  female
1312    3rd      NaN    male

[1313 rows x 3 columns]


In [3]:
# 缺失值处理
x['age'].fillna(x['age'].mean(), inplace=True) # 用均值填充

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=4)
print(x_train.head())

/tmp/ipykernel_20326/4049615488.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x['age'].fillna(x['age'].mean(), inplace=True) # 用均值填充
/tmp/ipykernel_20326/4049615488.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['age'].fillna(x['age'].mean(), inplace=True) # 用均值填充


    pclass        age     sex
598    2nd  30.000000    male
246    1st  62.000000    male
905    3rd  31.194181  female
300    1st  31.194181  female
509    2nd  64.000000    male


In [4]:
from sklearn.feature_extraction import DictVectorizer

# 将非数值特征转换为 one-hot 编码
dict = DictVectorizer(sparse=False)

# to_dict 可以将 DataFrame 转换为字典列表
x_train = dict.fit_transform(x_train.to_dict(orient='records'))

print(type(x_train))
print('=================')
print(dict.get_feature_names_out())

x_test = dict.transform(x_test.to_dict(orient='records'))
print(x_train)

<class 'numpy.ndarray'>
['age' 'pclass=1st' 'pclass=2nd' 'pclass=3rd' 'sex=female' 'sex=male']
[[30.          0.          1.          0.          0.          1.        ]
 [62.          1.          0.          0.          0.          1.        ]
 [31.19418104  0.          0.          1.          1.          0.        ]
 ...
 [34.          0.          1.          0.          0.          1.        ]
 [46.          1.          0.          0.          0.          1.        ]
 [31.19418104  0.          0.          1.          0.          1.        ]]


In [5]:
from sklearn.tree import DecisionTreeClassifier
dec = DecisionTreeClassifier()  # 这是用于分类的决策树

# 训练
dec.fit(x_train, y_train)

print('训练集里面有多少男的：', x_train[:, 5].sum())
print('训练集里面有多少女的：', x_train[:, 4].sum())

print('预测的准确率：', dec.score(x_test, y_test))

# 导出决策树的结构
from sklearn.tree import export_graphviz
export_graphviz(dec, out_file="tree.dot",
                feature_names=dict.get_feature_names_out())

训练集里面有多少男的： 643.0
训练集里面有多少女的： 341.0
预测的准确率： 0.8085106382978723


这里会生成一个 ``tree.dot`` 文件，需要安装：

```shell
sudo apt install graphviz
```

然后执行命令：

```shell
dot -Tpng tree.dot -o tree.png
```

就得到下图：

![tree.png](./tree.png)

上图中，``value = [死亡， 存活]`` .

观察发现，即使是走到叶子节点，是否存活也是不一定的。这是因为，**为了防止过拟合，进行了剪枝**。

# 剪枝

## 为什么要剪枝

![pic5](./5.png)

随着树的增长，在训练样集上的精度是单调上升的，然而在独立的测试样例上测出的精度先上升后下降。

出现这种情况的原因：
- 噪声、样本冲突，即错误的样本数据。
- 特征即属性不能完全作为分类标准。
- 巧合的规律性，数据量不够大。

## 常用的剪枝方法

**预剪枝**：
- 每一个结点所包含的最小样本数目，例如10，则该结点总样本数小于10时，则不再分；
  - 参数 ``min_samples_split``
- 指定树的高度或者深度，例如树的最大深度为4；
- 指定结点的熵小于某个值，不再划分。随着树的增大，在训练样集上的精度是单调上升的，然而在独立的测试样例上测出的精度先上升后下降。

**后剪枝**:
在已生成的过拟合决策树上剪枝。

# 集成学习方法-随机森林

集成学习通过建立几个模型的组合来解决单一预测问题。它的工作原理是生成多个分类器/模型，各自独立地学习和作出预测。这些预测最后结合成单预测，因此优于任何一个单分类的做出预测。

在机器学习中，**随机森林**是一个包含多个决策树的分类器，其输出的类别由个别树输出的类别的**众数**而定。

例如，如果训练了 5 个树, 其中 4 个树结果是 True, 1 个树结果是 False, 那么最终结果是 True.

随机森林的优点：
- 在当前所有算法中，具有极好的准确率
- 能够有效地运行在大数据集上
- 能够处理具有高维特征的输入样本，而且不需要降维
  - 参数 ``max_features="auto”``, 每个决策树的最大特征数量
- 能够评估各个特征在分类问题上的重要性
- 对于缺省值问题也能够获得很好得结果

## 实例

In [ ]:
# 随机森林进行预测（超参数调优）
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# n_jobs=-1 是使用所有的 CPU 核心
rf = RandomForestClassifier(n_jobs = -1)

param = {
    'n_estimators': [50, 80],
    'max_depth': [2, 3, 5, 8, 15, 25]
}

gc = GridSearchCV(rf, param_grid = param, cv=3)

gc.fit(x_train, y_train)

print("准确率：", gc.score(x_test, y_test))
print("查看选择的参数：", gc.best_params_)
print("选择的最好模型是：", gc.best_estimator_)
print("每个超参数每次交叉验证的结果：", gc.cv_results_)

准确率： 0.8389057750759878
查看选择的参数： {'max_depth': 3, 'n_estimators': 80}
选择的最好模型是： RandomForestClassifier(max_depth=3, n_estimators=80, n_jobs=-1)
每个超参数每次交叉验证的结果： {'mean_fit_time': array([0.14392352, 0.24342028, 0.17605678, 0.24809376, 0.16642634,
       0.23049982, 0.1252439 , 0.26372814, 0.17696961, 0.2391026 ,
       0.12453755, 0.13311084]), 'std_fit_time': array([0.03557314, 0.01736664, 0.02824466, 0.01137561, 0.01475437,
       0.01895239, 0.01013709, 0.07819231, 0.02562576, 0.03062173,
       0.0305696 , 0.01633324]), 'mean_score_time': array([0.03291988, 0.05922087, 0.03991501, 0.05492504, 0.03968541,
       0.04200427, 0.03020398, 0.0559086 , 0.04339806, 0.04944158,
       0.03474371, 0.02569795]), 'std_score_time': array([0.0138835 , 0.00043513, 0.00693334, 0.0077229 , 0.00574088,
       0.00651898, 0.01138553, 0.00132847, 0.00874411, 0.01721119,
       0.01435718, 0.00102845]), 'param_max_depth': masked_array(data=[2, 2, 3, 3, 5, 5, 8, 8, 15, 15, 25, 25],
             mask=[Fal

## AIGC（有限验证）

随机森林使用的是**有放回的抽样**（Bootstrap抽样）。具体来说：

- 在训练每棵决策树时，随机森林会从原始训练数据集中**有放回地随机抽取样本**，生成一个与原始数据集大小相同的子集（称为Bootstrap样本）。
- 由于是有放回的抽样，部分样本可能会被多次抽取，而部分样本可能不会被抽取到（大约有 1/3 的样本未被抽取，称为袋外样本，Out-of-Bag Samples）。
- 每棵决策树只使用其对应的 Bootstrap 样本进行训练。

这种有放回的抽样方式是随机森林的核心之一，它通过引入样本的随机性，增加了模型的多样性，从而提高了整体的泛化能力。

`n_estimators` 参数在 `RandomForestClassifier` 中**并不是指样本的数量**，而是指随机森林中**决策树的数量**。这是一个常见的**误解**。

**解释**：
1. **`n_estimators` 的定义**：
   - 在 `sklearn.ensemble.RandomForestClassifier` 中，`n_estimators` 参数控制随机森林中包含的决策树的数量。
   - 每棵决策树会独立训练，并通过集成（如投票或平均）来生成最终的预测结果。

2. **样本的数量与决策树的关系**：
   - 每棵决策树的训练数据是从原始训练集通过**有放回的抽样**（Bootstrap抽样）生成的。
   - 样本的数量等于原始训练集的大小，但每棵树的样本是随机抽取的，且允许重复。

3. **误解的来源**：
   - 可能有人将 `n_estimators` 与样本数量混淆，因为每棵树的训练样本是从原始数据集中抽取的，但 `n_estimators` 实际上是控制森林中树的数量，而不是样本的数量。

在代码中：
- `n_estimators` 候选值是 `50` 和 `80`，表示随机森林中分别包含 50 棵树或 80 棵树。
- 这些树会独立训练，并通过集成方法（如投票）生成最终的预测结果。

根据 [scikit-learn 官方文档](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)，`n_estimators` 的定义是：
> The number of trees in the forest.

因此，`n_estimators` 是决策树的数量，而不是样本的数量。